# Local Field Potential (LFP) Analysis

## Overview

This notebook provides a comprehensive guide to analyzing Local Field Potentials (LFPs) using Python. We will cover the entire pipeline from data loading and preprocessing to advanced analyses such as time-frequency analysis, coherence, and phase-amplitude coupling (PAC). The methods used here are commonly applied to understand oscillatory dynamics, neuronal interactions, and synchronization in the brain.

### Objectives
- Load and preprocess LFP data.
- Perform time-frequency and coherence analysis.
- Compute phase-amplitude coupling (PAC) metrics.
- Visualize results for better interpretation.

### Required Libraries
- `neo`
- `elephant`
- `matplotlib`
- `plotly`
- `numpy`
- `lfp_analysis` (custom module)

Ensure these are installed before proceeding.

## Prerequisites and Installation

Before running the notebook, ensure you have all the required libraries installed. If you haven't installed them yet, you can use the commands below.

In [ ]:
# Python Cell: Install necessary libraries
%pip install neo elephant matplotlib plotly numpy

## Data Loading and Inspection

The first step in LFP analysis is to load the raw data. We will use `neo` to handle data formats commonly used in neuroscience, such as Neuralynx or NeuroExplorer files. The data is then converted into a format compatible with `elephant` for further analysis.


In [ ]:
# Python Cell: Load data
from lfp_analysis import load_data

# Provide the path to your LFP data file
file_path = 'data/sample_lfp_data'  # Replace with the actual data path

# Load the LFP data
recording = load_data(file_path)

# Inspect the data
print(f"Data shape: {recording.get_traces().shape}")
print(f"Sampling rate: {recording.get_sampling_frequency()} Hz")


## Preprocessing

To improve the signal-to-noise ratio, we preprocess the LFP data using bandpass and notch filters. This step reduces noise and removes powerline interference.

In [ ]:
# Python Cell: Preprocess data
from lfp_analysis import preprocess_data

# Apply preprocessing: bandpass filter (1-100 Hz) and notch filter at 50 Hz
preprocessed_data = preprocess_data(recording, freq_min=1, freq_max=100, notch_freq=50)

# Plot the preprocessed signal
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.plot(preprocessed_data.get_times(), preprocessed_data.get_traces()[0])
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (mV)')
plt.title('Preprocessed LFP Signal')
plt.show()

## Time-Frequency Analysis

Time-frequency analysis provides insights into the oscillatory components of the LFP signals. We use the Welch method to compute the power spectral density (PSD).

In [ ]:
# Python Cell: Time-Frequency Analysis
from lfp_analysis import time_frequency_analysis

# Perform time-frequency analysis
freqs, psd = time_frequency_analysis(preprocessed_data)

# Plot Power Spectral Density (PSD)
import plotly.express as px

fig = px.line(x=freqs, y=psd, labels={'x': 'Frequency (Hz)', 'y': 'Power Spectral Density (PSD)'},
              title='Power Spectral Density (PSD) of LFP')
fig.show()

## Coherence Analysis

Coherence analysis quantifies the synchrony between two signals, which can provide insights into the functional connectivity between brain regions.


In [ ]:
# Python Cell: Coherence Analysis
from lfp_analysis import coherence_analysis

# Assume we are comparing the same signal for simplicity in demonstration
coherency, freqs_coherence = coherence_analysis(preprocessed_data, preprocessed_data)

# Plot Coherence
fig = px.line(x=freqs_coherence, y=coherency, labels={'x': 'Frequency (Hz)', 'y': 'Coherence'},
              title='Coherence Analysis between Signals')
fig.show()

## Phase-Amplitude Coupling (PAC) Analysis

PAC analysis measures how the phase of a lower frequency oscillation modulates the amplitude of a higher frequency oscillation, revealing cross-frequency interactions.

In [ ]:
# Python Cell: PAC Analysis
from lfp_analysis import pac_analysis

# Calculate PAC between theta (4-8 Hz) phase and gamma (30-100 Hz) amplitude
pac_value = pac_analysis(preprocessed_data, (4, 8), (30, 100))
print(f"PAC Modulation Index: {pac_value}")

## Visualization of Results

We will visualize the results using interactive plots provided by Plotly and static plots using Matplotlib.

In [ ]:
# Python Cell: Visualizations
from lfp_analysis import plot_power_spectral_density, plot_coherence, plot_pac

# Plot PSD
plot_power_spectral_density(freqs, psd)

# Plot Coherence
plot_coherence(freqs_coherence, coherency)

# Plot PAC
plot_pac(pac_value)

## Conclusion and Future Directions

In this notebook, we covered the steps for performing LFP analysis, including preprocessing, time-frequency analysis, coherence analysis, and phase-amplitude coupling (PAC). These methods help in understanding the oscillatory dynamics and connectivity in the brain.

### Future Directions
- Explore more advanced connectivity measures like Granger Causality or Dynamic Causal Modeling.
- Integrate machine learning techniques for automatic classification of LFP patterns.
- Apply these analyses to different brain states or conditions to investigate neural correlates.